#  Import Required Libraries


In [89]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import time

##  Define Target URL


In [90]:
url ="https://www.shiksha.com/engineering/colleges/b-tech-colleges-india" 

## Send HTTP Request with Custom Headers


In [91]:
headers ={
"User-Agent": "Mozilla/5.0 (Linux; Android 11; SAMSUNG SM-G973U) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/14.2 Chrome/87.0.4280.141 Mobile Safari/537.36"
} 

response= requests.get(url,headers=headers) 

## Check HTTP Response Status

In [92]:
print(response) 

<Response [200]>


## Parse HTML Content with BeautifulSoup


In [93]:
soup =BeautifulSoup(response.content,parser="html")
print(soup.prettify()) 

<!DOCTYPE html>
<html class="mobile" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <title id="seoTitle">
   Top BTech Colleges in India: Ranking, Fees, Courses, Admission 2025, Placements, Cutoff
  </title>
  <meta content="B.E. / B.Tech colleges in India, B.E. / B.Tech in India, B.E. / B.Tech courses in India, list of B.E. / B.Tech colleges in India, B.E. / B.Tech colleges India, colleges in India for B.E. / B.Tech " name="keywords"/>
  <meta content="Find 4816 best Engineering Colleges in India for B.Tech. Compare Fees, Courses, Placements, Cut off, Student Reviews and Admission process." id="metaDescription" name="description"/>
  <link href="https://www.shiksha.com/engineering/colleges/b-tech-colleges-india" id="canonicalUrl" rel="canonical"/>
  <link href="https://www.shiksha.com/engineering/colleges/b-tech-colleges-india-2" rel="next"/>
  <meta content="Top BTech Colleges in India: Ranking, Fees, Courses, Admissio

## Loop Through Pages and Scrape Data  
- Define base URL and headers  
- Create a Empty list for storing data (`all_colleges`, `location`, `slot`)  
- Written loop for iterating the multiple pages 
- Construct page-specific URL according the page url 
- Send HTTP request and parse response with BeautifulSoup  
- Extract:  
  - College Names  
  - Locations  
  - Information Box (Details)  
- Append extracted data into lists  
-  Add delay to avoid overloading the server  


In [94]:
# Base URL pattern
base_url = "https://www.shiksha.com/engineering/colleges/b-tech-colleges-india"

# Custom headers
headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 11; SAMSUNG SM-G973U) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/14.2 Chrome/87.0.4280.141 Mobile Safari/537.36"
}

all_colleges = []
location =[]
slot=[] 

# loop iterating over each page 
for page in range(0, 200):   
    if page == 0:
        url = base_url
    else:
        url = f"{base_url}-{page}"
    
    print(f"Scraping page {page+1}: {url}")
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # get all college name 
    colleges = soup.find_all("h3", class_="f7cc")
    for college in colleges:
        all_colleges.append(college.text)

    # get all location :    
    loc = soup.find_all('span',class_="_5588")
    for i in loc :
        location.append(i.text)
        
    # get the information box
    section = soup.find_all("div",class_="cd4f _5c64 contentColumn_2")
    for i in section:
        slot.append(i.text)
        
        
        
    time.sleep(2)  # polite delay



Scraping page 1: https://www.shiksha.com/engineering/colleges/b-tech-colleges-india
Scraping page 2: https://www.shiksha.com/engineering/colleges/b-tech-colleges-india-1
Scraping page 3: https://www.shiksha.com/engineering/colleges/b-tech-colleges-india-2
Scraping page 4: https://www.shiksha.com/engineering/colleges/b-tech-colleges-india-3
Scraping page 5: https://www.shiksha.com/engineering/colleges/b-tech-colleges-india-4
Scraping page 6: https://www.shiksha.com/engineering/colleges/b-tech-colleges-india-5
Scraping page 7: https://www.shiksha.com/engineering/colleges/b-tech-colleges-india-6
Scraping page 8: https://www.shiksha.com/engineering/colleges/b-tech-colleges-india-7
Scraping page 9: https://www.shiksha.com/engineering/colleges/b-tech-colleges-india-8
Scraping page 10: https://www.shiksha.com/engineering/colleges/b-tech-colleges-india-9
Scraping page 11: https://www.shiksha.com/engineering/colleges/b-tech-colleges-india-10
Scraping page 12: https://www.shiksha.com/engineering

## Verify Extracted Data Length


In [95]:
print(len(all_colleges))
print(len(location))
print(len(slot))

2541
2541
2541


## Create DataFrame and Display Scraped Data

In [96]:
df = pd.DataFrame({"College Name":all_colleges,"Location":location,"slot":slot})
print(df)

                                           College Name         Location  \
0                                           VIT Vellore          Vellore   
1           IIT Bombay - Indian Institute of Technology           Mumbai   
2                  DTU - Delhi Technological University            Delhi   
3            IIT Delhi - Indian Institute of Technology            Delhi   
4     BITS Pilani - Birla Institute of Technology an...           Pilani   
...                                                 ...              ...   
2536  BITS Pilani - Birla Institute of Technology an...           Pilani   
2537        IIT Madras - Indian Institute of Technology          Chennai   
2538  SRM Institute of Science and Technology, Katta...          Chennai   
2539      NIT Trichy - National Institute of Technology  Tiruchirappalli   
2540               LPU - Lovely Professional University        Jalandhar   

                                                   slot  
0     Courses Offered26 Cours

## Check Number of Duplicate Rows in DataFrame

In [97]:
df.duplicated().sum() 

403

## Remove Duplicate Rows from DataFrame


In [98]:
df.drop_duplicates(inplace=True) 

## Check Missing Values in Each Column


In [99]:
df.isna().sum()

College Name    0
Location        0
slot            0
dtype: int64

## Import Regular Expressions (re) Module


In [100]:
import re

## Extract Information from Text  
- Make a function to pick details using regex:  
  - Number of courses  
  - College rating  
  - Exams accepted  
  - Tuition fees  
  - Placement rating or salary  
- Apply this function on the "slot" column  
- Save results in new columns  
- Show the final clean DataFrame  


In [101]:
def extract_info(text):
    # 1. No of Courses
    courses = re.search(r"Courses Offered(\d+)", text)
    courses = courses.group(1) if courses else None

    # 2. College rating
    rating = re.search(r"Courses Offered\d+\s*Courses([\d.]+)", text)
    rating = rating.group(1) if rating else None

    # 3. Exams Accepted
    exams = re.search(r"Exams Accepted(.*?)Total Tuition Fees", text)
    exams = exams.group(1).strip() if exams else None

    # 4. Tuition Fees
    fees = re.search(r"Total Tuition Fees(₹[\d.,\s\-\+A-ZKLP]+)", text)
    fees = fees.group(1).strip() if fees else None

    # 5. Placement Rating OR Median Salary
    placement_or_salary = None
    placement_search = re.search(r"(Placement Rating[\d.]+|Median Salary₹[\d.,\s\-\+A-ZKLP]+)", text)
    if placement_search:
        placement_or_salary = placement_search.group(1).strip()

    return pd.Series([courses, rating, exams, fees, placement_or_salary])

# Apply extraction
df[["No_Of_Course_Offer", "College_rating", "Exam_Accepted", 
    "Tuition_Fees", "Placement_or_Salary"]] = df["slot"].apply(extract_info)

# Final clean DataFrame
print(df[["No_Of_Course_Offer","College_rating","Exam_Accepted",
          "Tuition_Fees","Placement_or_Salary"]])

     No_Of_Course_Offer College_rating                  Exam_Accepted  \
0                    26            4.2                         VITEEE   
1                    25            4.6           JEE MainJEE Advanced   
2                    20            4.3              JEE MainJAC Delhi   
3                    17            4.5           JEE MainJEE Advanced   
4                    10            4.4                         BITSAT   
...                 ...            ...                            ...   
2195                  4            3.6                  JEE MainUPTAC   
2196                  5           None                      AP EAMCET   
2197                  7           None                  JEE MainUPTAC   
2198                  6           None  CBSE 12thISCTamilnadu 12th +1   
2222                 36            4.6           JEE MainJEE Advanced   

             Tuition_Fees              Placement_or_Salary  
0        ₹4.87 L - 7.8 LP              Placement Rating4.2  
1

## Show DataFrame


In [102]:
df

,College Name,Location,slot,No_Of_Course_Offer,College_rating,Exam_Accepted,Tuition_Fees,Placement_or_Salary
0,VIT Vellore,Vellore,Courses Offered26 Courses4.2Exams AcceptedVITE...,26,4.2,VITEEE,₹4.87 L - 7.8 LP,Placement Rating4.2
1,IIT Bombay - Indian Institute of Technology,Mumbai,Courses Offered25 Courses4.6Exams AcceptedJEE ...,25,4.6,JEE MainJEE Advanced,₹8 L - 10 LM,Median Salary₹17.92 - 19.63 LPA
2,DTU - Delhi Technological University,Delhi,Courses Offered20 Courses4.3Exams AcceptedJEE ...,20,4.3,JEE MainJAC Delhi,₹5.32 L - 7.4 LP,Placement Rating4.4
3,IIT Delhi - Indian Institute of Technology,Delhi,Courses Offered17 Courses4.5Exams AcceptedJEE ...,17,4.5,JEE MainJEE Advanced,₹8 L - 8.35 LM,Median Salary₹20 - 22 LPA
4,BITS Pilani - Birla Institute of Technology an...,Pilani,Courses Offered10 Courses4.4Exams AcceptedBITS...,10,4.4,BITSAT,₹20.76 L - 61.49 LM,Median Salary₹16.15 LPA
...,...,...,...,...,...,...,...,...
2195,Rishi Chadha Vishvas Girls Institute of Techno...,Ghaziabad,Courses Offered4 Courses3.6Exams AcceptedJEE M...,4,3.6,JEE MainUPTAC,₹2.45 LP,Placement Rating2.0
2196,"Priyadarshini College Of Engineering, Sullurpet",Nellore,Courses Offered5 CoursesExams AcceptedAP EAMCE...,5,None,AP EAMCET,₹1.8 LM,None
2197,Saroj Institute of Management & Technology,Lucknow,Courses Offered7 CoursesExams AcceptedJEE Main...,7,None,JEE MainUPTAC,₹2.45 LM,None
2198,Oxford College of Engineering,Tiruvannamalai,Courses Offered6 CoursesExams AcceptedCBSE 12t...,6,None,CBSE 12thISCTamilnadu 12th +1,₹2 LM,None


## Drop Unwanted Column 'slot' from DataFrame


In [103]:
df = df.drop("slot", axis=1)

## Display Updated DataFrame


In [104]:
df

,College Name,Location,No_Of_Course_Offer,College_rating,Exam_Accepted,Tuition_Fees,Placement_or_Salary
0,VIT Vellore,Vellore,26,4.2,VITEEE,₹4.87 L - 7.8 LP,Placement Rating4.2
1,IIT Bombay - Indian Institute of Technology,Mumbai,25,4.6,JEE MainJEE Advanced,₹8 L - 10 LM,Median Salary₹17.92 - 19.63 LPA
2,DTU - Delhi Technological University,Delhi,20,4.3,JEE MainJAC Delhi,₹5.32 L - 7.4 LP,Placement Rating4.4
3,IIT Delhi - Indian Institute of Technology,Delhi,17,4.5,JEE MainJEE Advanced,₹8 L - 8.35 LM,Median Salary₹20 - 22 LPA
4,BITS Pilani - Birla Institute of Technology an...,Pilani,10,4.4,BITSAT,₹20.76 L - 61.49 LM,Median Salary₹16.15 LPA
...,...,...,...,...,...,...,...
2195,Rishi Chadha Vishvas Girls Institute of Techno...,Ghaziabad,4,3.6,JEE MainUPTAC,₹2.45 LP,Placement Rating2.0
2196,"Priyadarshini College Of Engineering, Sullurpet",Nellore,5,None,AP EAMCET,₹1.8 LM,None
2197,Saroj Institute of Management & Technology,Lucknow,7,None,JEE MainUPTAC,₹2.45 LM,None
2198,Oxford College of Engineering,Tiruvannamalai,6,None,CBSE 12thISCTamilnadu 12th +1,₹2 LM,None


## Clean Placement_or_Salary Column  
- Remove text "Placement Rating"  
- Remove text "Median Salary"  



In [105]:
df["Placement_or_Salary"] = df["Placement_or_Salary"].str.replace("Placement Rating", "", regex=False)
df["Placement_or_Salary"] = df["Placement_or_Salary"].str.replace("Median Salary", "", regex=False)
df["Placement_or_Salary"] = df["Placement_or_Salary"].str.strip()   # remove extra spaces

## Fix Exam_Accepted Column  
- Add space or slash between exams  


In [106]:
df["Exam_Accepted"] = df["Exam_Accepted"].str.replace(r"(?<=[a-z])(?=[A-Z])", " / ", regex=True)

## Show Cleaned DataFrame


In [107]:
df

,College Name,Location,No_Of_Course_Offer,College_rating,Exam_Accepted,Tuition_Fees,Placement_or_Salary
0,VIT Vellore,Vellore,26,4.2,VITEEE,₹4.87 L - 7.8 LP,4.2
1,IIT Bombay - Indian Institute of Technology,Mumbai,25,4.6,JEE Main / JEE Advanced,₹8 L - 10 LM,₹17.92 - 19.63 LPA
2,DTU - Delhi Technological University,Delhi,20,4.3,JEE Main / JAC Delhi,₹5.32 L - 7.4 LP,4.4
3,IIT Delhi - Indian Institute of Technology,Delhi,17,4.5,JEE Main / JEE Advanced,₹8 L - 8.35 LM,₹20 - 22 LPA
4,BITS Pilani - Birla Institute of Technology an...,Pilani,10,4.4,BITSAT,₹20.76 L - 61.49 LM,₹16.15 LPA
...,...,...,...,...,...,...,...
2195,Rishi Chadha Vishvas Girls Institute of Techno...,Ghaziabad,4,3.6,JEE Main / UPTAC,₹2.45 LP,2.0
2196,"Priyadarshini College Of Engineering, Sullurpet",Nellore,5,None,AP EAMCET,₹1.8 LM,None
2197,Saroj Institute of Management & Technology,Lucknow,7,None,JEE Main / UPTAC,₹2.45 LM,None
2198,Oxford College of Engineering,Tiruvannamalai,6,None,CBSE 12th / ISCTamilnadu 12th +1,₹2 LM,None


## Update Tuition_Fees Column  
- Replace short forms (L, LP, LM, KP) with "Lakh"  


In [108]:
df["Tuition_Fees"]=df["Tuition_Fees"].str.replace(r"\bL\b|\bLP\b|\bLM\b|\bKP\b", "Lakh", regex=True)

## View DataFrame After Cleaning


In [109]:
df

,College Name,Location,No_Of_Course_Offer,College_rating,Exam_Accepted,Tuition_Fees,Placement_or_Salary
0,VIT Vellore,Vellore,26,4.2,VITEEE,₹4.87 Lakh - 7.8 Lakh,4.2
1,IIT Bombay - Indian Institute of Technology,Mumbai,25,4.6,JEE Main / JEE Advanced,₹8 Lakh - 10 Lakh,₹17.92 - 19.63 LPA
2,DTU - Delhi Technological University,Delhi,20,4.3,JEE Main / JAC Delhi,₹5.32 Lakh - 7.4 Lakh,4.4
3,IIT Delhi - Indian Institute of Technology,Delhi,17,4.5,JEE Main / JEE Advanced,₹8 Lakh - 8.35 Lakh,₹20 - 22 LPA
4,BITS Pilani - Birla Institute of Technology an...,Pilani,10,4.4,BITSAT,₹20.76 Lakh - 61.49 Lakh,₹16.15 LPA
...,...,...,...,...,...,...,...
2195,Rishi Chadha Vishvas Girls Institute of Techno...,Ghaziabad,4,3.6,JEE Main / UPTAC,₹2.45 Lakh,2.0
2196,"Priyadarshini College Of Engineering, Sullurpet",Nellore,5,None,AP EAMCET,₹1.8 Lakh,None
2197,Saroj Institute of Management & Technology,Lucknow,7,None,JEE Main / UPTAC,₹2.45 Lakh,None
2198,Oxford College of Engineering,Tiruvannamalai,6,None,CBSE 12th / ISCTamilnadu 12th +1,₹2 Lakh,None


## Count Exams  
- Make a function to count exams  
- Handle empty values  
- Count exams using "/"  
- Add extra exams from "+number"  
- Save result in new column  


In [ ]:
import re

def count_exams(text):
    if pd.isnull(text) or text.strip() in ["–", "– / –"]:
        return 0
    
    # Base count = number of '/' + 1
    base_count = text.count("/") + 1
    
    # Check if '+N' exists (e.g., +3, +5)
    match = re.search(r"\+(\d+)", text)
    extra = int(match.group(1)) if match else 0
    
    return base_count + extra

# Apply fix
df["exam_count"] = df["exams_accepted"].apply(count_exams)

In [167]:

def count_exams(text):
    if pd.isnull(text) or text.strip() in ["–", "– / –"]:
        return 1

    count = text.count("/") + 1
    
    match = re.search(r"\+(\d+)", text)
    if match:
        count += int(match.group(1))
    
    return count

df["Exam_Count"] = df["exams_accepted"].apply(count_exams)


In [168]:
df


,college_name,location,num_courses_offered,college_rating,exams_accepted,exam_count,min_fees,max_fees,avg_fees,placement_rating,min_package,max_package,avg_package,Exam_Count
0,VIT Vellore,Vellore,26,4.2,VITEEE,1,4.87,7.80,6.34,4.2,NaN,NaN,NaN,1
1,IIT Bombay - Indian Institute of Technology,Mumbai,25,4.6,JEE Main / JEE Advanced,2,8.00,10.00,9.00,4.6,17.92,19.63,18.77,2
2,DTU - Delhi Technological University,Delhi,20,4.3,JEE Main / JAC Delhi,2,5.32,7.40,6.36,4.4,NaN,NaN,NaN,2
3,IIT Delhi - Indian Institute of Technology,Delhi,17,4.5,JEE Main / JEE Advanced,2,8.00,8.35,8.18,4.5,20.00,22.00,21.00,2
4,BITS Pilani - Birla Institute of Technology an...,Pilani,10,4.4,BITSAT,1,20.76,61.49,41.12,4.4,16.15,16.15,16.15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,Modern Institute of Engineering and Technology...,Kurukshetra,6,2.7,JEE Main / HSTES,2,2.04,2.31,2.17,2.3,NaN,NaN,NaN,2
1712,Gian Jyoti Group of Institutions,Patiala,6,2.5,JEE Main / PTU BTech,2,2.42,3.18,2.80,1.0,NaN,NaN,NaN,2
1713,Lord Krishna College of Engineering (LKCE),Ghaziabad,5,2.1,CBSE 12th / UP 12th / ISC +2,5,2.45,2.45,2.45,1.3,NaN,NaN,NaN,5
1714,Rishi Chadha Vishvas Girls Institute of Techno...,Ghaziabad,4,3.6,JEE Main / UPTAC,2,2.45,2.45,2.45,2.0,NaN,NaN,NaN,2


## Show All Column Names of DataFrame

In [114]:
df.columns 

Index(['College Name', 'Location', 'No_Of_Course_Offer', 'College_rating',
       'Exam_Accepted', 'Tuition_Fees', 'Placement_or_Salary', 'Exam_Count'],
      dtype='object')

## Show DataFrame Info (column types, non-null values, memory usage)

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2138 entries, 0 to 2222
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   College Name         2138 non-null   object
 1   Location             2138 non-null   object
 2   No_Of_Course_Offer   2138 non-null   object
 3   College_rating       1695 non-null   object
 4   Exam_Accepted        2138 non-null   object
 5   Tuition_Fees         1940 non-null   object
 6   Placement_or_Salary  1799 non-null   object
 7   Exam_Count           2138 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 150.3+ KB


## Check Missing Values in Each Column


In [116]:
df.isnull().sum()

College Name             0
Location                 0
No_Of_Course_Offer       0
College_rating         443
Exam_Accepted            0
Tuition_Fees           198
Placement_or_Salary    339
Exam_Count               0
dtype: int64

## Checking the Shape

In [117]:
df.shape


(2138, 8)

## Check percentage of missing values

In [118]:
(df.isnull().sum()/len(df))*100

College Name            0.000000
Location                0.000000
No_Of_Course_Offer      0.000000
College_rating         20.720299
Exam_Accepted           0.000000
Tuition_Fees            9.260992
Placement_or_Salary    15.855940
Exam_Count              0.000000
dtype: float64

## Handle Missing Values  
- Missing values are very few in "Tuition_Fees" and "Placement_or_Salary"  
- Drop rows with missing values in these columns  


In [119]:
df.dropna(subset=['Tuition_Fees','Placement_or_Salary'],inplace=True)
df.shape

(1716, 8)

- filling null values with median of college_ratings 

In [120]:
df['College_rating']=df['College_rating'].astype(float)
df["College_rating"].fillna(df["College_rating"].median(), inplace=True)



C:\Users\JAY SUDHIR KADU\AppData\Local\Temp\ipykernel_38792\831209181.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["College_rating"].fillna(df["College_rating"].median(), inplace=True)


## Check Missing Values in Each Column

In [121]:
df.isnull().sum()

College Name           0
Location               0
No_Of_Course_Offer     0
College_rating         0
Exam_Accepted          0
Tuition_Fees           0
Placement_or_Salary    0
Exam_Count             0
dtype: int64

## Typecasting of necessary column 
-  No_Of_Course_Offer object into int 

In [122]:
df['No_Of_Course_Offer']=df['No_Of_Course_Offer'].astype(int)

In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1716 entries, 0 to 2222
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   College Name         1716 non-null   object 
 1   Location             1716 non-null   object 
 2   No_Of_Course_Offer   1716 non-null   int32  
 3   College_rating       1716 non-null   float64
 4   Exam_Accepted        1716 non-null   object 
 5   Tuition_Fees         1716 non-null   object 
 6   Placement_or_Salary  1716 non-null   object 
 7   Exam_Count           1716 non-null   int64  
dtypes: float64(1), int32(1), int64(1), object(5)
memory usage: 114.0+ KB


## Spliting the "Tuition_Fees" column in two columns which are min_fees and max_fees 
- cleaning the min_fees column and type casting in float
- cleaning the max_fees column and type casting in float 

In [124]:
df[['Min_fees','Max_fees']]=df['Tuition_Fees'].str.split('-',expand=True)

In [125]:
df['Min_fees']=df['Min_fees'].str.replace("₹","").str.replace("Lakh","").str.replace('M',"").str.replace("K","")

In [126]:
df['Min_fees']=df['Min_fees'].astype(float)

In [127]:
df['Max_fees']=df['Max_fees'].str.replace("Lakh","").str.replace('KM','')
df['Max_fees']=df['Max_fees'].astype(float)

In [128]:
df.isna().sum()

College Name             0
Location                 0
No_Of_Course_Offer       0
College_rating           0
Exam_Accepted            0
Tuition_Fees             0
Placement_or_Salary      0
Exam_Count               0
Min_fees                 0
Max_fees               867
dtype: int64

- filling the "Max_fees" null value with the same "min_fees"
  

In [129]:
df['Max_fees'].fillna(df['Min_fees'],inplace=True)

C:\Users\JAY SUDHIR KADU\AppData\Local\Temp\ipykernel_38792\637964654.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Max_fees'].fillna(df['Min_fees'],inplace=True)


In [130]:
df['Max_fees'].isna().sum()

0

In [131]:
df['Avg_fees']=((df['Min_fees']+df["Max_fees"])/2).round(2)

In [132]:
df

,College Name,Location,No_Of_Course_Offer,College_rating,Exam_Accepted,Tuition_Fees,Placement_or_Salary,Exam_Count,Min_fees,Max_fees,Avg_fees
0,VIT Vellore,Vellore,26,4.2,VITEEE,₹4.87 Lakh - 7.8 Lakh,4.2,1,4.87,7.80,6.34
1,IIT Bombay - Indian Institute of Technology,Mumbai,25,4.6,JEE Main / JEE Advanced,₹8 Lakh - 10 Lakh,₹17.92 - 19.63 LPA,2,8.00,10.00,9.00
2,DTU - Delhi Technological University,Delhi,20,4.3,JEE Main / JAC Delhi,₹5.32 Lakh - 7.4 Lakh,4.4,2,5.32,7.40,6.36
3,IIT Delhi - Indian Institute of Technology,Delhi,17,4.5,JEE Main / JEE Advanced,₹8 Lakh - 8.35 Lakh,₹20 - 22 LPA,2,8.00,8.35,8.18
4,BITS Pilani - Birla Institute of Technology an...,Pilani,10,4.4,BITSAT,₹20.76 Lakh - 61.49 Lakh,₹16.15 LPA,1,20.76,61.49,41.12
...,...,...,...,...,...,...,...,...,...,...,...
2186,Modern Institute of Engineering and Technology...,Kurukshetra,6,2.7,JEE Main / HSTES,₹2.04 Lakh - 2.31 Lakh,2.3,2,2.04,2.31,2.17
2192,Gian Jyoti Group of Institutions,Patiala,6,2.5,JEE Main / PTU BTech,₹2.42 Lakh - 3.18 Lakh,1.0,2,2.42,3.18,2.80
2194,Lord Krishna College of Engineering (LKCE),Ghaziabad,5,2.1,CBSE 12th / UP 12th / ISC +2,₹2.45 Lakh,1.3,5,2.45,2.45,2.45
2195,Rishi Chadha Vishvas Girls Institute of Techno...,Ghaziabad,4,3.6,JEE Main / UPTAC,₹2.45 Lakh,2.0,2,2.45,2.45,2.45


In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1716 entries, 0 to 2222
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   College Name         1716 non-null   object 
 1   Location             1716 non-null   object 
 2   No_Of_Course_Offer   1716 non-null   int32  
 3   College_rating       1716 non-null   float64
 4   Exam_Accepted        1716 non-null   object 
 5   Tuition_Fees         1716 non-null   object 
 6   Placement_or_Salary  1716 non-null   object 
 7   Exam_Count           1716 non-null   int64  
 8   Min_fees             1716 non-null   float64
 9   Max_fees             1716 non-null   float64
 10  Avg_fees             1716 non-null   float64
dtypes: float64(4), int32(1), int64(1), object(5)
memory usage: 154.2+ KB


In [134]:
df.head(20)


,College Name,Location,No_Of_Course_Offer,College_rating,Exam_Accepted,Tuition_Fees,Placement_or_Salary,Exam_Count,Min_fees,Max_fees,Avg_fees
0,VIT Vellore,Vellore,26,4.2,VITEEE,₹4.87 Lakh - 7.8 Lakh,4.2,1,4.87,7.80,6.34
1,IIT Bombay - Indian Institute of Technology,Mumbai,25,4.6,JEE Main / JEE Advanced,₹8 Lakh - 10 Lakh,₹17.92 - 19.63 LPA,2,8.00,10.00,9.00
2,DTU - Delhi Technological University,Delhi,20,4.3,JEE Main / JAC Delhi,₹5.32 Lakh - 7.4 Lakh,4.4,2,5.32,7.40,6.36
3,IIT Delhi - Indian Institute of Technology,Delhi,17,4.5,JEE Main / JEE Advanced,₹8 Lakh - 8.35 Lakh,₹20 - 22 LPA,2,8.00,8.35,8.18
4,BITS Pilani - Birla Institute of Technology an...,Pilani,10,4.4,BITSAT,₹20.76 Lakh - 61.49 Lakh,₹16.15 LPA,1,20.76,61.49,41.12
5,"SRM Institute of Science and Technology, Katta...",Chennai,55,4.0,JEE Main / SRMJEEE,₹11 Lakh - 23.75 Lakh,3.9,2,11.00,23.75,17.38
6,NIT Trichy - National Institute of Technology,Tiruchirappalli,9,4.3,JEE Main / DASA UG,₹5 Lakh,4.5,2,5.00,5.00,5.00
7,LPU - Lovely Professional University,Jalandhar,75,4.3,JEE Main / LPU NESTCUETCBSE 12th / ISCPSEB 12t...,₹8.4 Lakh - 16 Lakh,₹7.07 LPA,8,8.40,16.00,12.20
8,Chandigarh University (CU),Chandigarh,41,4.2,CBSE 12th / ISCPSEB 12th / CUCET Chandigarh Un...,₹5.7 Lakh - 14 Lakh,4.2,5,5.70,14.00,9.85
19,Kalasalingam Academy of Research and Education,Virudhunagar,18,3.9,JEE Main,₹4 Lakh - 7.4 Lakh,₹6.75 LPA,1,4.00,7.40,5.70


## Reset DataFrame Index  
- Reset the index after dropping rows  
- Old index is removed

In [135]:
df.reset_index(inplace=True)

## Split Placement_or_Salary Column  
- Create **Rating** column (keep values without salary info)  
- Create **Salary** column (keep values with ₹ or LPA)  


In [136]:
df["Rating"] = df["Placement_or_Salary"].where(
    ~df["Placement_or_Salary"].str.contains("₹|LPA", na=False),
    np.nan
)

df["Salary"] = df["Placement_or_Salary"].where(
    df["Placement_or_Salary"].str.contains("₹|LPA", na=False),
    np.nan
)

## Check Total Missing Values After Splitting Columns


In [138]:
df.isna().sum()

index                     0
College Name              0
Location                  0
No_Of_Course_Offer        0
College_rating            0
Exam_Accepted             0
Tuition_Fees              0
Placement_or_Salary       0
Exam_Count                0
Min_fees                  0
Max_fees                  0
Avg_fees                  0
Rating                  632
Salary                 1084
dtype: int64

## Fill Missing Ratings  
- Replace NaN in "Rating" with values from "College_rating"  


In [140]:
df['Rating'].fillna(df['College_rating'],inplace=True)

C:\Users\JAY SUDHIR KADU\AppData\Local\Temp\ipykernel_38792\724296689.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Rating'].fillna(df['College_rating'],inplace=True)


## Check Missing Values After Filling 'Rating' Column

In [173]:
df.isnull().sum()

college_name              0
location                  0
num_courses_offered       0
college_rating            0
exams_accepted            0
exam_count                0
min_fees                  0
max_fees                  0
avg_fees                  0
placement_rating          0
min_package            1084
max_package            1084
avg_package            1084
Exam_Count                0
dtype: int64

## Split Salary Column  
- Create **Min_salary** and **Max_salary** by splitting Salary column  
- Remove "₹" and "LPA" from values  


In [142]:
df[['Min_salary','Max_salary']]=df['Salary'].str.split('-',expand=True)

In [143]:
df['Min_salary']=df['Min_salary'].str.replace('₹',"").str.replace("LPA","")

In [144]:
df['Max_salary']=df['Max_salary'].str.replace("LPA","")

## Display Cleaned DataFrame


In [146]:
df

,index,College Name,Location,No_Of_Course_Offer,College_rating,Exam_Accepted,Tuition_Fees,Placement_or_Salary,Exam_Count,Min_fees,Max_fees,Avg_fees,Rating,Salary,Min_salary,Max_salary
0,0,VIT Vellore,Vellore,26,4.2,VITEEE,₹4.87 Lakh - 7.8 Lakh,4.2,1,4.87,7.80,6.34,4.2,NaN,NaN,None
1,1,IIT Bombay - Indian Institute of Technology,Mumbai,25,4.6,JEE Main / JEE Advanced,₹8 Lakh - 10 Lakh,₹17.92 - 19.63 LPA,2,8.00,10.00,9.00,4.6,₹17.92 - 19.63 LPA,17.92,19.63
2,2,DTU - Delhi Technological University,Delhi,20,4.3,JEE Main / JAC Delhi,₹5.32 Lakh - 7.4 Lakh,4.4,2,5.32,7.40,6.36,4.4,NaN,NaN,None
3,3,IIT Delhi - Indian Institute of Technology,Delhi,17,4.5,JEE Main / JEE Advanced,₹8 Lakh - 8.35 Lakh,₹20 - 22 LPA,2,8.00,8.35,8.18,4.5,₹20 - 22 LPA,20,22
4,4,BITS Pilani - Birla Institute of Technology an...,Pilani,10,4.4,BITSAT,₹20.76 Lakh - 61.49 Lakh,₹16.15 LPA,1,20.76,61.49,41.12,4.4,₹16.15 LPA,16.15,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,2186,Modern Institute of Engineering and Technology...,Kurukshetra,6,2.7,JEE Main / HSTES,₹2.04 Lakh - 2.31 Lakh,2.3,2,2.04,2.31,2.17,2.3,NaN,NaN,None
1712,2192,Gian Jyoti Group of Institutions,Patiala,6,2.5,JEE Main / PTU BTech,₹2.42 Lakh - 3.18 Lakh,1.0,2,2.42,3.18,2.80,1.0,NaN,NaN,None
1713,2194,Lord Krishna College of Engineering (LKCE),Ghaziabad,5,2.1,CBSE 12th / UP 12th / ISC +2,₹2.45 Lakh,1.3,5,2.45,2.45,2.45,1.3,NaN,NaN,None
1714,2195,Rishi Chadha Vishvas Girls Institute of Techno...,Ghaziabad,4,3.6,JEE Main / UPTAC,₹2.45 Lakh,2.0,2,2.45,2.45,2.45,2.0,NaN,NaN,None


## Fill Missing Max_salary  
- If Max_salary is missing, replace it with Min_salary  


In [147]:
df['Max_salary'].fillna(df['Min_salary'],inplace=True)

C:\Users\JAY SUDHIR KADU\AppData\Local\Temp\ipykernel_38792\3290388175.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Max_salary'].fillna(df['Min_salary'],inplace=True)


## Check Missing Values After Filling Max_salary


In [148]:
df.isnull().sum()

index                     0
College Name              0
Location                  0
No_Of_Course_Offer        0
College_rating            0
Exam_Accepted             0
Tuition_Fees              0
Placement_or_Salary       0
Exam_Count                0
Min_fees                  0
Max_fees                  0
Avg_fees                  0
Rating                    0
Salary                 1084
Min_salary             1084
Max_salary             1084
dtype: int64

## Displaying Dataframe

In [149]:
df

,index,College Name,Location,No_Of_Course_Offer,College_rating,Exam_Accepted,Tuition_Fees,Placement_or_Salary,Exam_Count,Min_fees,Max_fees,Avg_fees,Rating,Salary,Min_salary,Max_salary
0,0,VIT Vellore,Vellore,26,4.2,VITEEE,₹4.87 Lakh - 7.8 Lakh,4.2,1,4.87,7.80,6.34,4.2,NaN,NaN,NaN
1,1,IIT Bombay - Indian Institute of Technology,Mumbai,25,4.6,JEE Main / JEE Advanced,₹8 Lakh - 10 Lakh,₹17.92 - 19.63 LPA,2,8.00,10.00,9.00,4.6,₹17.92 - 19.63 LPA,17.92,19.63
2,2,DTU - Delhi Technological University,Delhi,20,4.3,JEE Main / JAC Delhi,₹5.32 Lakh - 7.4 Lakh,4.4,2,5.32,7.40,6.36,4.4,NaN,NaN,NaN
3,3,IIT Delhi - Indian Institute of Technology,Delhi,17,4.5,JEE Main / JEE Advanced,₹8 Lakh - 8.35 Lakh,₹20 - 22 LPA,2,8.00,8.35,8.18,4.5,₹20 - 22 LPA,20,22
4,4,BITS Pilani - Birla Institute of Technology an...,Pilani,10,4.4,BITSAT,₹20.76 Lakh - 61.49 Lakh,₹16.15 LPA,1,20.76,61.49,41.12,4.4,₹16.15 LPA,16.15,16.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,2186,Modern Institute of Engineering and Technology...,Kurukshetra,6,2.7,JEE Main / HSTES,₹2.04 Lakh - 2.31 Lakh,2.3,2,2.04,2.31,2.17,2.3,NaN,NaN,NaN
1712,2192,Gian Jyoti Group of Institutions,Patiala,6,2.5,JEE Main / PTU BTech,₹2.42 Lakh - 3.18 Lakh,1.0,2,2.42,3.18,2.80,1.0,NaN,NaN,NaN
1713,2194,Lord Krishna College of Engineering (LKCE),Ghaziabad,5,2.1,CBSE 12th / UP 12th / ISC +2,₹2.45 Lakh,1.3,5,2.45,2.45,2.45,1.3,NaN,NaN,NaN
1714,2195,Rishi Chadha Vishvas Girls Institute of Techno...,Ghaziabad,4,3.6,JEE Main / UPTAC,₹2.45 Lakh,2.0,2,2.45,2.45,2.45,2.0,NaN,NaN,NaN


## Convert Salary Columns to Float  
- Change Min_salary and Max_salary to float type  


In [150]:
df['Min_salary']=df['Min_salary'].astype(float)

In [151]:
df['Max_salary']=df['Max_salary'].astype(float)

## Calculate Average Salary  
- Create new column Salary_Avg as the mean of Min_salary and Max_salary  
- Round values to 2 decimals  


In [153]:
df['Salary_Avg']=((df['Min_salary']+df['Max_salary'])/2).round(2)

## Display First 20 Rows of DataFrame


In [154]:
df.head(20)

,index,College Name,Location,No_Of_Course_Offer,College_rating,Exam_Accepted,Tuition_Fees,Placement_or_Salary,Exam_Count,Min_fees,Max_fees,Avg_fees,Rating,Salary,Min_salary,Max_salary,Salary_Avg
0,0,VIT Vellore,Vellore,26,4.2,VITEEE,₹4.87 Lakh - 7.8 Lakh,4.2,1,4.87,7.80,6.34,4.2,NaN,NaN,NaN,NaN
1,1,IIT Bombay - Indian Institute of Technology,Mumbai,25,4.6,JEE Main / JEE Advanced,₹8 Lakh - 10 Lakh,₹17.92 - 19.63 LPA,2,8.00,10.00,9.00,4.6,₹17.92 - 19.63 LPA,17.92,19.63,18.77
2,2,DTU - Delhi Technological University,Delhi,20,4.3,JEE Main / JAC Delhi,₹5.32 Lakh - 7.4 Lakh,4.4,2,5.32,7.40,6.36,4.4,NaN,NaN,NaN,NaN
3,3,IIT Delhi - Indian Institute of Technology,Delhi,17,4.5,JEE Main / JEE Advanced,₹8 Lakh - 8.35 Lakh,₹20 - 22 LPA,2,8.00,8.35,8.18,4.5,₹20 - 22 LPA,20.00,22.00,21.00
4,4,BITS Pilani - Birla Institute of Technology an...,Pilani,10,4.4,BITSAT,₹20.76 Lakh - 61.49 Lakh,₹16.15 LPA,1,20.76,61.49,41.12,4.4,₹16.15 LPA,16.15,16.15,16.15
5,5,"SRM Institute of Science and Technology, Katta...",Chennai,55,4.0,JEE Main / SRMJEEE,₹11 Lakh - 23.75 Lakh,3.9,2,11.00,23.75,17.38,3.9,NaN,NaN,NaN,NaN
6,6,NIT Trichy - National Institute of Technology,Tiruchirappalli,9,4.3,JEE Main / DASA UG,₹5 Lakh,4.5,2,5.00,5.00,5.00,4.5,NaN,NaN,NaN,NaN
7,7,LPU - Lovely Professional University,Jalandhar,75,4.3,JEE Main / LPU NESTCUETCBSE 12th / ISCPSEB 12t...,₹8.4 Lakh - 16 Lakh,₹7.07 LPA,8,8.40,16.00,12.20,4.3,₹7.07 LPA,7.07,7.07,7.07
8,8,Chandigarh University (CU),Chandigarh,41,4.2,CBSE 12th / ISCPSEB 12th / CUCET Chandigarh Un...,₹5.7 Lakh - 14 Lakh,4.2,5,5.70,14.00,9.85,4.2,NaN,NaN,NaN,NaN
9,19,Kalasalingam Academy of Research and Education,Virudhunagar,18,3.9,JEE Main,₹4 Lakh - 7.4 Lakh,₹6.75 LPA,1,4.00,7.40,5.70,3.9,₹6.75 LPA,6.75,6.75,6.75


## Show DataFrame Information (columns, datatypes, non-null counts, memory usage)


In [155]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716 entries, 0 to 1715
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                1716 non-null   int64  
 1   College Name         1716 non-null   object 
 2   Location             1716 non-null   object 
 3   No_Of_Course_Offer   1716 non-null   int32  
 4   College_rating       1716 non-null   float64
 5   Exam_Accepted        1716 non-null   object 
 6   Tuition_Fees         1716 non-null   object 
 7   Placement_or_Salary  1716 non-null   object 
 8   Exam_Count           1716 non-null   int64  
 9   Min_fees             1716 non-null   float64
 10  Max_fees             1716 non-null   float64
 11  Avg_fees             1716 non-null   float64
 12  Rating               1716 non-null   object 
 13  Salary               632 non-null    object 
 14  Min_salary           632 non-null    float64
 15  Max_salary           632 non-null    f

## Drop Unwanted 'index' Column from DataFrame


In [159]:
df.drop("index",axis=1,inplace=True)

In [175]:
df

,college_name,location,num_courses_offered,college_rating,exams_accepted,exam_count,min_fees,max_fees,avg_fees,placement_rating,min_package,max_package,avg_package,Exam_Count
0,VIT Vellore,Vellore,26,4.2,VITEEE,1,4.87,7.80,6.34,4.2,NaN,NaN,NaN,1
1,IIT Bombay - Indian Institute of Technology,Mumbai,25,4.6,JEE Main / JEE Advanced,2,8.00,10.00,9.00,4.6,17.92,19.63,18.77,2
2,DTU - Delhi Technological University,Delhi,20,4.3,JEE Main / JAC Delhi,2,5.32,7.40,6.36,4.4,NaN,NaN,NaN,2
3,IIT Delhi - Indian Institute of Technology,Delhi,17,4.5,JEE Main / JEE Advanced,2,8.00,8.35,8.18,4.5,20.00,22.00,21.00,2
4,BITS Pilani - Birla Institute of Technology an...,Pilani,10,4.4,BITSAT,1,20.76,61.49,41.12,4.4,16.15,16.15,16.15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,Modern Institute of Engineering and Technology...,Kurukshetra,6,2.7,JEE Main / HSTES,2,2.04,2.31,2.17,2.3,NaN,NaN,NaN,2
1712,Gian Jyoti Group of Institutions,Patiala,6,2.5,JEE Main / PTU BTech,2,2.42,3.18,2.80,1.0,NaN,NaN,NaN,2
1713,Lord Krishna College of Engineering (LKCE),Ghaziabad,5,2.1,CBSE 12th / UP 12th / ISC +2,5,2.45,2.45,2.45,1.3,NaN,NaN,NaN,5
1714,Rishi Chadha Vishvas Girls Institute of Techno...,Ghaziabad,4,3.6,JEE Main / UPTAC,2,2.45,2.45,2.45,2.0,NaN,NaN,NaN,2


## Rename DataFrame Columns  
- Assign clear and consistent names to all columns  


In [161]:
df.columns = [
    "college_name",
    "location",
    "num_courses_offered",
    "college_rating",        
    "exams_accepted",
    "exam_count",
    "min_fees",
    "max_fees",
    "avg_fees",
    "placement_rating",      
    "min_package",          
    "max_package",           
    "avg_package"            
]

## Clean exams_accepted Column  
- Replace "JEE Main / JEE Advanced" with "JEE Advanced"  


In [171]:
df['exams_accepted'].str.replace("JEE Main / JEE Advanced","JEE Advanced")

0                                                VITEEE
1                                          JEE Advanced
2                                  JEE Main / JAC Delhi
3                                          JEE Advanced
4                                                BITSAT
5                                    JEE Main / SRMJEEE
6                                    JEE Main / DASA UG
7     JEE Main / LPU NESTCUETCBSE 12th / ISCPSEB 12t...
8     CBSE 12th / ISCPSEB 12th / CUCET Chandigarh Un...
9                                              JEE Main
10    MHT CETJEE Main / PERA CETCBSE 12th / ISCMahar...
11        JEE Main / METJEE Advanced / CUETSAT India +3
12          JEE Main / CBSE 12th / ISCUK 12th / CUET +4
13                                   JEE Main / DASA UG
14                   JEE Main / UPESEATCUETSAT India +3
15           JEE Main / AP EAMCETJEE Advanced / VSAT +3
16                   JEE Main / SAT India / CUETNEET +3
17                         CUETCBSE 12th / ISCWB

## Save DataFrame to CSV File  
- Export cleaned DataFrame as 'college_data.csv' without index  



In [165]:
df.to_csv('college_data.csv',index=False)  